<a href="https://colab.research.google.com/github/rkes/JupyterNoteBooks/blob/main/CS6890_assignment1_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy import stats
import pandas as pd
import numpy as np
import math
from scipy.sparse.linalg import svds
#import surprise
from scipy.sparse import coo_matrix
from numpy.linalg import norm
from sklearn.metrics import mean_squared_error
from numpy import linalg as LA

In [4]:
cols=["user_id","movie_id","ratings","timestamp"]
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/u1.base",sep="\t",names=cols,encoding="latin-1")
test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/u1.test",sep="\t",names=cols,encoding="latin-1")
ratings_pivot = df.pivot(index='movie_id', columns='user_id', values="ratings").fillna(0)
test_ratings_pivot = df.pivot(index='movie_id', columns='user_id', values="ratings").fillna(0)

test_df_group_by_user = test_df.groupby('user_id')
test_df_group_by_movie = test_df.groupby('movie_id')

train_df_user_movie = df.pivot(
    index='movie_id',
    columns='user_id',
    values='ratings'
).fillna(0)


test_df_user_movie = test_df.pivot(
    index='movie_id',
    columns='user_id',
    values='ratings'
).fillna(0)

#Creating the rating matrix (rows as movies, columns as users)
ratings_mat = np.ndarray(
    shape=(np.max(df.movie_id.values), np.max(df.user_id.values)),
    dtype=np.float)
ratings_mat[df.movie_id.values-1, df.user_id.values-1] = df.ratings.values.astype(float)



samples = [
            (i, j, ratings_mat[i, j])
            for i in range(1,df.movie_id.max())
            for j in range(1,df.user_id.max())
            if ratings_mat[i, j] > 0
        ]



In [5]:
def SGD(R, K, lamda=0.005,steps=10):
    
    M,N = R.shape
    # According to slide i have initialized P and Q by SVD
    P,U,Q=svds(R,K)
    minErr = error(R,P,Q,lamda);
    for step in range(steps):
        for ui in range(len(R.data)):
            rui=R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            if rui>0:
                eui = 2*(rui - np.dot(P[u,:],Q[:,i]))
                if math.isnan(eui):
                    P[u,:]=P[u,:]+lamda*eui*(Q[:,i])
                    Q[:,i]=Q[:,i]+lamda*eui*(P[u,:])
        err = error(R,P,Q,lamda);
        rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
        if rmse<0.5:
            break
        if err > minErr:
            break;
        minErr = err
    return P,Q

def error(R,P,Q,lamda=0.05):
    ratings = R.data
    rows = R.row
    cols = R.col
    e = 0 
    for ui in range(len(ratings)):
        rui=ratings[ui]
        u = rows[ui]
        i = cols[ui]
        if rui>0:
            e= e + pow(rui-np.dot(P[u,:],Q[:,i]),2)
    return e

def get_group(group,key):
    if key in group.groups: return group.get_group(key)
    return pd.DataFrame()
    

def SGDWithRegularization(R, K, lamda=0.005,steps=10,L1=0.01,L2=0.03,gamma=0.001):
    
    M,N = R.shape
    # According to slide i have initialized P and Q by SVD
    P,U,Q=svds(R,K)
    minErr = np.zeros(df.user_id.max()+1) 
    for i in range(df.user_id.max()+1):
        minErr[i] = np.Inf;
    #hl, = plt.plot([], [])  
    for step in range(steps):
        px =0;
        qx =0;
        print("Step {}".format(step))
        for ui in range(len(R.data)):
            rui=R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            if rui>0:
                eui = 2*(rui - np.dot(P[u,:],Q[:,i]))
                #print(" ** ",eui,"  ",step," user  ",u)
                #P[u,:]=P[u,:]+lamda*eui*(Q[:,i])
                #Q[:,i]=Q[:,i]+lamda*eui*(P[u,:])
                
                tempP=P[u,:]+lamda*eui*Q[:,i]
                tempQ=Q[:,i]+lamda*eui*P[u,:]
                tempPx = LA.norm(P[u,:])
                tempQx= LA.norm(Q[:,i])
                 #qx += LA.norm(Q[:,i])
                err = error(R,P,Q,lamda);
                if  minErr[i] > err:
                    P[u,:]=tempP
                    Q[:,i]=tempQ
                    px +=  tempPx
                    qx +=  tempQx
                    minErr[i]=err
                    
                rmse = np.sqrt(err/len(R.data))
                #print("px,qx,rmse "," ** ",px,qx,rmse)
        #print(px,qx)
        err = error(R,P,Q,lamda);
        rmse = np.sqrt(err/len(R.data))
        err = err + L1*px+L2*qx # added Regularization
        if rmse<0.5:
            break;
        if err > minErr:
            break;
        minErr = err
    return P,Q


In [7]:
def caculateRatingOnLatentFactor(res,key):
    for i in list(test_df_group_by_movie.groups.keys()):
        for j in list(test_df_group_by_user.groups.keys()):
            test_df.loc[((test_df["movie_id"]==i) & (test_df["user_id"]==j)),key] =res[i,j]

def getAvgPrecisionAt10(key):
    threashold=4
    grpDfCalc = test_df.sort_values([key]).groupby('user_id')
    group_by_key = 'movie_id';
    actual_data = ratings_pivot.values;
    cnt=0;
    total =0;
    for mov in actual_data:
            mov_idx_ratingGtThreashold = np.argwhere(mov>threashold)
            for userKey in list(grpDfCalc.groups.keys()):
                user_movies = grpDfCalc.get_group(userKey)
                union = len(np.union1d(mov_idx_ratingGtThreashold,user_movies));
                intersection = len(np.intersect1d(mov_idx_ratingGtThreashold,user_movies));
                if union>0 and intersection>0:          
                    total+=(intersection/union);
                cnt+=1;
    if total>0:
        return (total/cnt)
    return 0;

In [ ]:
R = coo_matrix(ratings_pivot.values)
P,Q=SGDWithRegularization(R,K=2, steps=1,L1=0.001,L2=0.003)

Step 0
